#### Modified from:
https://plot.ly/python/lines-on-maps/

#### Improvements required:
- add state/countie boundaries
- click on Indiana county should highlight outlflow destinations
- on click each line should provide info on number of outflows


In [1]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd

In [2]:
df_locations = pd.read_excel('migration.xlsx', sheet_name='locations')
df_locations.head()

,fips,county_label,state_id,state_label,xcoord,ycoord,location
0,18001,Adams County,IN,Indiana,-84.933837,40.762354,"-84.9338373165914, 40.7623542013296"
1,18003,Allen County,IN,Indiana,-85.064557,41.104480,"-85.064556610331, 41.1044797878032"
2,18005,Bartholomew County,IN,Indiana,-85.911729,39.200573,"-85.9117293170047, 39.2005731136601"
3,18007,Benton County,IN,Indiana,-87.317880,40.612965,"-87.3178798806813, 40.6129654495163"
4,18009,Blackford County,IN,Indiana,-85.323221,40.491468,"-85.3232205245655, 40.4914681340455"


In [3]:
?df_migration.reset_index

Object `df_migration.reset_index` not found.


In [4]:
df_migration = pd.read_excel('migration.xlsx', sheet_name='county_outflow')

#Filter aggregate and non-migration reccords
df_migration = df_migration[pd.notnull(df_migration['destination_xcoord'])].query('non_migration == 0')
df_migration.reset_index(inplace=True, drop=True)
df_migration.head()

,origin_state,origin_county,origin_fips,origin_xcoord,origin_ycoord,origin_county_label,destination_state,destination_county,destination_fips,destination_xcoord,destination_ycoord,destination_state_lable,destination_county_label,non_migration,returns,excemptions,sum
0,18,1,18001,-84.933837,40.762354,Adams County,18,3,18003,-85.064557,41.104480,IN,Allen County,0,150.0,260.0,410.0
1,18,1,18001,-84.933837,40.762354,Adams County,18,179,18179,-85.226163,40.746742,IN,Wells County,0,87.0,158.0,245.0
2,18,1,18001,-84.933837,40.762354,Adams County,18,75,18075,-84.998554,40.451857,IN,Jay County,0,35.0,84.0,119.0
3,18,1,18001,-84.933837,40.762354,Adams County,39,161,39161,-84.575624,40.854753,OH,Van Wert County,0,21.0,51.0,72.0
4,18,3,18003,-85.064557,41.104480,Allen County,18,97,18097,-86.149632,39.784499,IN,Marion County,0,399.0,647.0,1046.0


In [32]:
counties = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_locations['xcoord'],
        lat = df_locations['ycoord'],
        hoverinfo = 'text',
        text = df_locations['county_label'] + "<br>" +  df_locations['state_label'],
        mode = 'markers',
        marker = dict( 
            size=3, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=5,
                color='rgba(68, 68, 68, 0)'
            )
        ))]

In [33]:
migration_paths = []
for i in range(len(df_migration)):
    migration_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [df_migration['origin_xcoord'][i], df_migration['destination_xcoord'][i]],
            lat = [df_migration['origin_ycoord'][i], df_migration['destination_ycoord'][i]],
            mode = 'lines',
            line = dict(
                width = 2,
                color = 'red',
            ),
            opacity = float(df_migration['sum'][i])/float(df_migration['sum'].max()),
        )
    )

In [37]:
layout = dict(
        title = 'Outflow Migration from Indiana Counties',
        showlegend = False, 
        geo = dict(
            scope='north america',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            showlakes = True,
            showocean = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            oceancolor = 'rgb(65, 105, 225)',
            lakecolor  = 'rgb(65, 105, 225)'
        ),
    )

In [38]:
fig = dict(data=migration_paths + counties, layout=layout )

In [39]:
plot(fig, filename='indiana-county-outflow.html' )

'file://C:\\Users\\axelm\\Documents\\GitHub\\migration-map\\indiana-county-outflow.html'